In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

In [2]:
%run "./technical_analysis_self.ipynb"

In [3]:
script_name = 'tcs'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 365)

In [4]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

In [5]:
for fn in [load_volume_sma, load_emas, load_macd, load_bollinger_bands, load_rsi, load_ATR, load_ADX]:
    df = fn(df)
    
df = load_aroon(df, 14)

In [6]:
df.tail()

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,...,smooth_DXm,plus_DI,minus_DI,DX,ADX,period,days_since_period_high,days_since_period_low,aroon_up,aroon_down
26,2022-09-12,TCS,3269.80,3225.0,3239.90,3242.95,3238.0,3217.65,1785928,4043.0,...,13.965337,28.784560,23.680718,9.728038,18.523345,14,13.0,9.0,7.142857,35.714286
27,2022-09-13,TCS,3263.45,3225.0,3263.45,3229.35,3233.0,3242.95,1972678,4043.0,...,12.967813,27.409874,22.549780,9.728038,17.895109,14,13.0,10.0,7.142857,28.571429
28,2022-09-14,TCS,3141.40,3113.8,3135.00,3120.40,3125.0,3229.35,5090269,4043.0,...,19.984398,23.740500,32.414112,15.445949,17.720169,14,13.0,11.0,7.142857,21.428571
1,2022-09-15,TCS,3137.75,3100.0,3130.00,3104.35,3105.5,3120.40,2138023,4043.0,...,19.542655,22.672629,32.600432,17.961376,17.737398,14,3.0,12.0,78.571429,14.285714
2,2022-09-16,TCS,3094.35,3000.0,3076.00,3008.70,3015.8,3104.35,5322221,4043.0,...,25.289609,19.995218,40.067370,33.418725,18.857493,14,4.0,0.0,71.428571,100.000000


In [7]:
initial_columns = list(df.columns)
df.columns

Index(['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close',
       'volume', 'high_52w', 'low_52w', 'vol_sma_10d', 'ema_9d', 'ema_21d',
       'ema_30d', 'ema_50d', 'ema_100d', 'ema_12d', 'ema_26d', 'macd',
       'macd_9d_signal', 'sma_20d', 'bb_upper', 'bb_lower', 'gain_pts',
       'loss_pts', 'gain_avg', 'loss_avg', 'rsi', 'true_range', 'ATR',
       'plus_DM', 'minus_DM', 'plus_DX', 'minus_DX', 'smooth_DXp',
       'smooth_DXm', 'plus_DI', 'minus_DI', 'DX', 'ADX', 'period',
       'days_since_period_high', 'days_since_period_low', 'aroon_up',
       'aroon_down'],
      dtype='object')

In [8]:
df["next_day_open"] = df["open"].shift(-1)

# Moving average crossovers

### 9d - 21d crossover

In [9]:
required_columns = ["date", "symbol", "high", "low", "open", "close", "prev_close", "next_day_open"]

In [10]:
df_9_21_co = df[
    ((df['ema_9d'] > df['ema_21d']) & (df['ema_9d'].shift(1) < df['ema_21d'].shift(1))) |
    (df['ema_9d'] < df['ema_21d']) & (df['ema_9d'].shift(1) > df['ema_21d'].shift(1))
][required_columns + ["ema_9d", "ema_21d"]]

In [11]:
df_9_21_co["signal"] = np.where(df_9_21_co["ema_9d"] > df_9_21_co["ema_21d"], "buy", "sell")
df_9_21_co["pct_returns"] = np.where(
    df_9_21_co["ema_21d"] > df_9_21_co["ema_9d"], 
    (df_9_21_co["next_day_open"].diff(1) / df_9_21_co['next_day_open'].shift(1)) * 100, 
    None
)

In [12]:
df_9_21_co

,date,symbol,high,low,open,close,prev_close,next_day_open,ema_9d,ema_21d,signal,pct_returns
232,2021-09-28,TCS,3850.0,3751.25,3850.0,3779.15,3836.95,3759.80,3835.947162,3840.800348,sell,NaN
239,2021-10-07,TCS,3899.0,3820.05,3830.0,3892.90,3810.90,3925.00,3820.988032,3819.123532,buy,None
241,2021-10-11,TCS,3757.5,3660.00,3755.0,3685.60,3935.65,3706.00,3812.003447,3815.917615,sell,-5.579618
221,2021-12-02,TCS,3648.0,3574.10,3589.2,3642.90,3577.80,3646.45,3532.775183,3522.556181,buy,None
148,2022-01-27,TCS,3733.4,3625.10,3731.0,3649.25,3769.90,3646.00,3805.923963,3807.336942,sell,-0.012341
153,2022-02-03,TCS,3882.5,3816.05,3851.0,3824.60,3856.20,3815.30,3800.401260,3800.303428,buy,None
155,2022-02-07,TCS,3831.8,3756.15,3791.0,3779.00,3814.90,3791.55,3798.440806,3799.573010,sell,-0.622494
128,2022-03-22,TCS,3711.0,3625.00,3640.0,3700.95,3626.70,3708.00,3638.770230,3637.115544,buy,None
93,2022-04-18,TCS,3610.0,3522.50,3610.0,3528.05,3661.95,3550.00,3668.119960,3683.798210,sell,-4.261057
53,2022-07-29,TCS,3317.3,3275.00,3290.0,3301.90,3260.50,3327.00,3195.059600,3185.236854,buy,None


### 30d - 50d crossover

In [13]:
df_30_50_co = df[
    ((df['ema_30d'] > df['ema_50d']) & (df['ema_30d'].shift(1) < df['ema_50d'].shift(1))) |
    ((df['ema_30d'] < df['ema_50d']) & (df['ema_30d'].shift(1) > df['ema_50d'].shift(1)))
][required_columns + ["ema_30d", "ema_50d"]]

In [14]:
df_30_50_co['signal'] = np.where(df_30_50_co['ema_30d'] > df_30_50_co['ema_50d'], 'buy', 'sell')
df_30_50_co['pct_returns'] = np.where(
    df_30_50_co['ema_30d'] < df_30_50_co['ema_50d'], 
    (df_30_50_co["next_day_open"].diff(1) / df_30_50_co['next_day_open'].shift(1)) * 100, 
    None
)

In [15]:
df_30_50_co

,date,symbol,high,low,open,close,prev_close,next_day_open,ema_30d,ema_50d,signal,pct_returns
232,2021-09-28,TCS,3850.00,3751.25,3850.0,3779.15,3836.95,3759.8,3841.726418,3842.527178,sell,NaN
240,2021-10-08,TCS,3989.90,3900.00,3925.0,3935.65,3892.90,3755.0,3831.524379,3830.554876,buy,None
241,2021-10-11,TCS,3757.50,3660.00,3755.0,3685.60,3935.65,3706.0,3817.172655,3818.530606,sell,-1.304927
181,2021-12-23,TCS,3670.50,3630.00,3648.0,3662.70,3630.75,3685.0,3579.391057,3578.922200,buy,None
168,2022-02-24,TCS,3483.85,3391.10,3474.0,3401.65,3563.80,3468.0,3727.644275,3728.011389,sell,-5.888738
138,2022-04-05,TCS,3835.60,3772.00,3772.0,3814.80,3770.35,3809.3,3696.462852,3693.493670,buy,None
93,2022-04-18,TCS,3610.00,3522.50,3610.0,3528.05,3661.95,3550.0,3684.650462,3687.156845,sell,-6.807025
11,2022-08-19,TCS,3421.50,3371.25,3387.0,3385.75,3381.25,3365.0,3301.125986,3299.845703,buy,None
15,2022-08-25,TCS,3278.15,3214.75,3276.0,3218.20,3255.35,3234.3,3294.901829,3296.317846,sell,-3.884101


# MACD - Signal line Crossover

In [16]:
df_macd = df[
    ((df['macd'] > df['macd_9d_signal']) & (df['macd'].shift(1) < df['macd_9d_signal'].shift(1))) |
    ((df['macd'] < df['macd_9d_signal']) & (df['macd'].shift(1) > df['macd_9d_signal'].shift(1)))
][required_columns + ['macd', 'macd_9d_signal']]

In [17]:
df_macd['signal'] = np.where(df_macd['macd'] > df_macd['macd_9d_signal'], 'buy', 'sell')
df_macd['pct_returns'] = np.where(
    df_macd['macd'] < df_macd['macd_9d_signal'], 
    (df_macd["next_day_open"].diff(1) / df_macd['next_day_open'].shift(1)) * 100, 
    None
)

In [18]:
df_macd

,date,symbol,high,low,open,close,prev_close,next_day_open,macd,macd_9d_signal,signal,pct_returns
231,2021-09-27,TCS,3904.00,3802.90,3900.0,3836.95,3871.30,3850.0,0.299426,0.925986,sell,NaN
239,2021-10-07,TCS,3899.00,3820.05,3830.0,3892.90,3810.90,3925.0,-0.102290,-4.149279,buy,None
241,2021-10-11,TCS,3757.50,3660.00,3755.0,3685.60,3935.65,3706.0,-3.194908,-2.821388,sell,-5.579618
206,2021-11-10,TCS,3519.70,3480.20,3498.0,3484.30,3505.70,3480.0,-51.959671,-52.832216,buy,None
146,2022-01-24,TCS,3849.65,3740.10,3840.0,3771.35,3833.50,3769.5,53.287137,65.104168,sell,8.318966
123,2022-03-14,TCS,3648.35,3580.00,3580.0,3643.00,3599.15,3659.0,-38.765586,-40.366596,buy,None
93,2022-04-18,TCS,3610.00,3522.50,3610.0,3528.05,3661.95,3550.0,-6.572940,6.528020,sell,-2.978956
70,2022-06-01,TCS,3388.00,3340.05,3388.0,3355.20,3364.35,3355.0,-62.734396,-66.235106,buy,None
80,2022-06-15,TCS,3234.00,3201.00,3218.5,3206.30,3210.55,3240.0,-47.626159,-45.036786,sell,-3.42772
87,2022-06-24,TCS,3345.00,3277.50,3339.0,3293.10,3308.80,3350.0,-47.783274,-54.193641,buy,None


# ADX Based Signals

In [19]:
df_adx = df[
    df["ADX"] > 20
][required_columns + ["plus_DI", "minus_DI", "DX", "ADX"]]

In [20]:
df_adx["signal"] = np.where(
    np.logical_or(
        np.logical_and(df_adx["plus_DI"] > df_adx["minus_DI"], df_adx["plus_DI"].shift(1) < df_adx["minus_DI"].shift(1)),
        np.logical_and(df_adx["plus_DI"] < df_adx["minus_DI"], df_adx["plus_DI"].shift(1) > df_adx["minus_DI"].shift(1)),
    ),
    np.where(df_adx[["plus_DI", "minus_DI"]].idxmax(axis=1) == 'plus_DI', "buy", "sell"),
    None
)

df_adx = df_adx[df_adx["signal"].notna()]

In [21]:
df_adx['returns'] = np.where(
    np.logical_and(
        df_adx['signal'] == "sell",
        df_adx['signal'].shift(1) == "buy"
    ),
    df_adx["close"].diff() * 100 / df_adx["close"].shift(1),
    None
)

In [22]:
df_adx[df_adx["signal"].notna()]

,date,symbol,high,low,open,close,prev_close,next_day_open,plus_DI,minus_DI,DX,ADX,signal,returns
232,2021-09-28,TCS,3850.00,3751.25,3850.00,3779.15,3836.95,3759.8,18.305462,27.610707,20.265724,48.648563,sell,None
239,2021-10-07,TCS,3899.00,3820.05,3830.00,3892.90,3810.90,3925.0,21.850803,17.215252,11.865930,32.300456,buy,None
241,2021-10-11,TCS,3757.50,3660.00,3755.00,3685.60,3935.65,3706.0,22.317535,35.601735,22.935718,31.598260,sell,-5.325079
220,2021-12-01,TCS,3590.00,3535.00,3535.00,3577.80,3529.15,3589.2,22.004116,19.526738,5.965152,26.079526,buy,None
223,2021-12-06,TCS,3658.00,3522.00,3640.45,3536.40,3640.45,3562.5,23.629108,26.157055,5.077609,23.915780,sell,-1.157136
225,2021-12-08,TCS,3646.45,3601.00,3601.00,3626.90,3584.50,3610.0,25.474391,22.589603,6.001974,21.339655,buy,None
146,2022-01-24,TCS,3849.65,3740.10,3840.00,3771.35,3833.50,3769.5,23.232491,24.992394,3.649367,28.676654,sell,3.98274
152,2022-02-02,TCS,3864.00,3800.65,3827.90,3856.20,3800.65,3851.0,25.721683,23.122468,5.321446,22.933808,buy,None
154,2022-02-04,TCS,3824.70,3769.00,3815.30,3814.90,3824.60,3791.0,24.537522,24.932726,0.798872,20.399469,sell,-1.071003
4,2022-08-08,TCS,3378.90,3336.00,3365.00,3374.45,3365.05,3385.0,29.520475,19.008487,21.661266,20.011903,buy,None


# Aroon

In [23]:
aroon_df = df[required_columns + ["aroon_up", "aroon_down"]]

In [24]:
aroon_df['signal'] = np.where(
    np.logical_or(
        np.logical_and(aroon_df["aroon_up"] > aroon_df["aroon_down"], aroon_df["aroon_up"].shift(1) < aroon_df["aroon_down"].shift(1)),
        np.logical_and(aroon_df["aroon_up"] < aroon_df["aroon_down"], aroon_df["aroon_up"].shift(1) > aroon_df["aroon_down"].shift(1)),
    ),
    np.where(aroon_df[["aroon_up", "aroon_down"]].idxmax(axis=1) == 'aroon_up', "buy", "sell"),
    None,
)

aroon_df = aroon_df[aroon_df.signal.notna()]

aroon_df['returns'] = np.where(
    aroon_df.signal == "sell",
    (aroon_df['next_day_open'] - aroon_df['next_day_open'].shift(1)) * 100 / aroon_df['next_day_open'].shift(1),
    None
)

<ipython-input-24-79891219f3e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aroon_df['signal'] = np.where(


In [25]:
aroon_df[aroon_df.signal.notna()]

,date,symbol,high,low,open,close,prev_close,next_day_open,aroon_up,aroon_down,signal,returns
240,2021-10-08,TCS,3989.90,3900.00,3925.00,3935.65,3892.90,3755.00,100.000000,71.428571,buy,None
241,2021-10-11,TCS,3757.50,3660.00,3755.00,3685.60,3935.65,3706.00,92.857143,100.000000,sell,-1.304927
208,2021-11-12,TCS,3534.15,3485.70,3506.00,3526.05,3488.20,3528.00,71.428571,35.714286,buy,None
214,2021-11-23,TCS,3480.95,3407.80,3445.00,3464.25,3458.40,3472.20,71.428571,100.000000,sell,-1.581633
220,2021-12-01,TCS,3590.00,3535.00,3535.00,3577.80,3529.15,3589.20,100.000000,85.714286,buy,None
178,2021-12-20,TCS,3606.00,3510.15,3589.00,3556.90,3584.35,3582.65,21.428571,100.000000,sell,-0.182492
181,2021-12-23,TCS,3670.50,3630.00,3648.00,3662.70,3630.75,3685.00,100.000000,78.571429,buy,None
145,2022-01-21,TCS,3851.55,3771.10,3807.00,3833.50,3826.55,3840.00,71.428571,100.000000,sell,4.206242
158,2022-02-10,TCS,3789.80,3758.00,3789.80,3770.35,3760.55,3752.50,64.285714,28.571429,buy,None
163,2022-02-17,TCS,3835.00,3779.00,3825.00,3784.20,3813.10,3769.00,28.571429,71.428571,sell,0.439707
